# Iterative Reconstruction

This script runs an iterative reconstruction of an in-situ hdf file or series of hdf files from the 2-BM instrument. Implementation here is designed for in-situ tests, reconstruction of ex-situ data takes a slightly different form. The pixel size must be changed from 0.875 um to 1.6 or 2.0 um. This can be optically verified in imagej or similar using the circle select tool and manually calculating it. <br>

The reconstruction must be chunked for RAM allocation, the `chunk_size` variable should be set correctly for this; `chunk_size` should be set to reflect the available RAM on the system. Running this without sufficient RAM will cause execution to fail.

In [1]:
import tomopy 
import tomocuda
import os
import numpy as np
from tomopy.recon.rotation import write_center
from tomopy.recon.rotation import find_center_vo
from tomopy.recon.algorithm import recon
from scipy import misc
import dxchange as tir
import datetime
import matplotlib.pyplot as plt


/opt/miniconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/opt/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Set some directory information. We need the directory, the input file and the flat and open beam files.

In [2]:
offset = 0
total_num_slices = 2000
chunk_size = 200
if chunk_size > total_num_slices:
    chunk_size = total_num_slices

margin_slices = 30
num_chunk = np.int(total_num_slices/chunk_size) + 1
if total_num_slices == chunk_size:
    num_chunk = 1
       
z = 10
eng = 27
pxl = 0.875

zinger_level = 200

data_dir  = '/run/media/james/37dd3227-85bf-4d9f-9575-ed621dc7c33b/raw/31_1day_7.5x_edge_75mm_1DegPerSec_180Deg_100msecExpTime_1500proj_Global_25umLuAG_1mmC_2mmGlass_25keV_2.657mrad_AHutch/'
file      = 'proj_200.hdf'
file_flat = 'proj_200.hdf'
file_dark = 'proj_200.hdf'

output_dir = data_dir
file_name = os.path.join(data_dir, file)
flat_name = os.path.join(data_dir, file_flat)
dark_name = os.path.join(data_dir, file_dark)
output_file = output_dir+'/recon_'+file.split(".")[-2]+'/recon_'+file.split(".")[-2] +'_'

Read data from the filesystem. The data are sliced here, we will work with a smaller data volume to get the centre of rotation of the specimen; this makes the operation faster as a reconstruction of the full stack is not required here.

In [3]:
data  = tir.read_hdf5(file_name, '/exchange/data',       slc=((1,1500), (0,2560,1)))
white = tir.read_hdf5(flat_name, '/exchange/data_dark',  slc=((1,9),    (0,2560,1)))
dark  = tir.read_hdf5(dark_name, '/exchange/data_white', slc=((1,9),    (0,2560,1)))

data_size = data.shape
theta = np.linspace(0,np.pi,num=data_size[0])

Take the data and open beam acquisitions, remove outliers from these. We do not remove outliers from the dark current results as these are inherently outliers. This is all done using the NVIDIA CUDA `tomocuda` toolkit. Placing load on the GPU makes the operation less CPU intensive.

In [ ]:
data  = tomopy.remove_outlier_cuda(data,zinger_level,size=15)
white = tomopy.remove_outlier_cuda(white,zinger_level,size=15)

In [ ]:
data = tomopy.prep.normalize.normalize(data,white,dark)
#data = tomopy.prep.normalize.normalize_bg(data,air=10)
#data[0,:,:] = data[1,:,:]

Stripes are removed using the Fourier wavelet method. These are quick operations and are done by the CPU,

In [1]:
data = tomopy.prep.stripe.remove_stripe_fw(data,level=6,wname='sym16',sigma=2,pad=True)

NameError: name 'tomopy' is not defined

Phase contrast is extracted and merged with the data using the ratio defined by ``rat``.

In [7]:
rat = 0.9
data = tomopy.prep.phase.retrieve_phase(data,pixel_size=pxl,dist=z,energy=eng,alpha=rat,pad=True)
data_size = data.shape

The centre of rotation is estimated as the mid-point of the image and adjusted relative to this. An isolated slice is reconstructed with a varying centre of rotation. The optimum centre is then extracted optically from the stack. This only needs to actually be run the first time around.

In [23]:
cs = data_size[2]/2-5
write_center(data[:,19:21,:], theta, dpath='/home/james/Data/Tomography/CoR/', cen_range=(2000, 2500, 10))
#write_center(data[:,9:11,:], theta, dpath='/home/james/Data/Tomography/CoR/', cen_range=(cs,cs+10,0.1))

In [9]:
def reconstruct(CR, time, infile, inflat, indark):
    center = CR
    
    out = output_file + str(time)
    for ii in range(num_chunk):
        if ii == 0:
            SliceStart = offset + ii     * chunk_size
            SliceEnd   = offset + (ii+1) * chunk_size
    
        else:
            SliceStart = offset + ii*(chunk_size-margin_slices)
            SliceEnd = offset + SliceStart + chunk_size
            if SliceEnd > (offset+total_num_slices):
                SliceEnd = offset+total_num_slices
        
        data_dir  = '/run/media/james/37dd3227-85bf-4d9f-9575-ed621dc7c33b/raw/PCBFS_91_longscan1_10x_dimax_75mm_36DegPerSec_180Deg_2msecExpTime_600proj_Rolling_100umLuAG_1mmC_2mmGlass_pink_2.657mrad_AHutch/'
        file_name = os.path.join(data_dir, infile)
        flat_name = os.path.join(data_dir, inflat)
        dark_name = os.path.join(data_dir, indark)
        
        data  = tir.read_hdf5(file_name,'/exchange/data_dark', slc=((1,600), (SliceStart,SliceEnd,1)))
        white = tir.read_hdf5(flat_name,'/exchange/data_dark', slc=((1,9),   (SliceStart,SliceEnd,1)))
        dark  = tir.read_hdf5(dark_name,'/exchange/data_dark', slc=((1,9),   (SliceStart,SliceEnd,1)))
        data_size = data.shape
        theta = np.linspace(0,np.pi,num=data_size[0])    
        
        # Damaged and/or corrupted images are replaced if 
        data[0,:,:] = data[1,:,:]
        
        # remove zingers (pixels with abnormal counts)
        data  = tomocuda.remove_outlier_cuda(data,  zinger_level, size=15)
        white = tomocuda.remove_outlier_cuda(white, zinger_level, size=15)
        
        # normalize projection images; for now you need to do below two operations in sequence
        data = tomopy.prep.normalize.normalize(data,white,dark)
        data = tomopy.prep.normalize.normalize_bg(data,air=10)
        
        # remove stripes in sinograms
        data = tomopy.prep.stripe.remove_stripe_fw(data,level=8,wname='sym16',sigma=1,pad=True)
        
        # phase retrieval
        data = tomopy.prep.phase.retrieve_phase(data,pixel_size=pxl,dist=z,energy=eng,alpha=rat,pad=True)
    
        # tomo reconstruction
        data_recon = recon(data, theta, center=center, algorithm='gridrec')
        
        # save reconstructions
        tir.writer.write_tiff_stack(data_recon[np.int(margin_slices/2):(SliceEnd-SliceStart-np.int(margin_slices/2)),:,:], 
                                                     axis = 0,
                                                     fname = out, 
                                                     start = SliceStart+np.int(margin_slices/2),
                                                     overwrite = True)

#times = [600,    1800,    3600,    5400,    7200,    21600,   28800,   43200]
ROT   = [1041.20]
#FILE  = ['proj_347.hdf', 'proj_356.hdf', 'proj_362.hdf', 'proj_371.hdf', 'proj_380.hdf', 'proj_452.hdf', 'proj_485.hdf','proj_560.hdf']
#FLAT  = ['proj_348.hdf', 'proj_357.hdf', 'proj_363.hdf', 'proj_372.hdf', 'proj_381.hdf', 'proj_453.hdf', 'proj_486.hdf','proj_561.hdf']
#DARK  = ['proj_349.hdf', 'proj_358.hdf', 'proj_364.hdf', 'proj_373.hdf', 'proj_382.hdf', 'proj_454.hdf', 'proj_487.hdf','proj_562.hdf']

times = [14400]
FILE  = ['proj_452.hdf']
FLAT  = ['proj_453.hdf']
DARK  = ['proj_454.hdf']

print("Total Iterations: " + str(len(ROT)))
for ii in range(1):#, #len(times)):
    print("Iteration " + str(ii + 1) + " Started... " + str(datetime.datetime.now()))
    reconstruct(ROT[0], times[ii], FILE[ii], FLAT[ii], DARK[ii])
    print("Iteration " + str(ii + 1) + " Ended... "   + str(datetime.datetime.now()))
    
print("Ended... " + str(datetime.datetime.now()))

Acquisition time at t0 20:03:09 <br>
Acquisition time at t1 20:36:56 <br>
Acquisition time at t2 20:59:26 <br>
Acquisition time at t3 21:33:10 <br>
Acquisition time at t4 22:06:54 <br>
Acquisition time at t5 23:59:26 <br>
Acquisition time at t6 04:50:54 <br>
Acquisition time at t7 09:21:19

In [10]:
range(1)[0]

0